In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import os
from tqdm.notebook import tqdm
from pathlib import Path
from rdkit import Chem

In [3]:
if os.path.exists("data/raw_chembl_affinity.parquet"):
    print("Raw ChEMBL affinity data already exists. Exiting to avoid overwriting.")
    df_sample = pd.read_parquet("data/raw_chembl_affinity.parquet")
    print(df_sample.shape)
else: 
    # Load raw data
    raw_data_dir = Path('data/chembl_parallel_enriched')
    activity_files = list(raw_data_dir.glob('uniprot_*/*_chembl_activities.parquet'))

    print(f"Found {len(activity_files)} UniProt activity files")
    print(f"Loading first 50 files for analysis...")

    # Load subset for faster analysis
    sample_activities = []
    for file in activity_files[:]:
        try:
            df = pd.read_parquet(file)
            sample_activities.append(df)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    df_sample = pd.concat(sample_activities, ignore_index=True)
    print(f"\nLoaded {len(df_sample):,} activities from {len(sample_activities)} files")
    print(f"DataFrame shape: {df_sample.shape}")
    df_sample['InchiKey'] = df_sample['canonical_smiles'].apply(
        lambda x: Chem.MolToInchiKey(Chem.MolFromSmiles(x)) if pd.notna(x) and Chem.MolFromSmiles(x) is not None else None
    )
    df_sample.to_parquet("data/raw_chembl_affinity.parquet")

    # Extract unique UniProt IDs from file paths
    unique_file_ids = sorted(list({f.parent.name.split('_')[1] for f in activity_files}))
    print(f"Found {len(unique_file_ids)} unique UniProt IDs in file structure")
    unique_file_ids[:10]

Raw ChEMBL affinity data already exists. Exiting to avoid overwriting.
(1237090, 73)


In [ ]:
if not os.path.exists("data/matched_annotation_table.parquet"):
    annot_df = pd.read_parquet('data/annotation_table_with_uniprot.parquet')
    # Filter annot_df to find entries associated with the UniProt IDs found in files
    unique_file_ids_set = set(unique_file_ids)
    # Count unique UniProt IDs
    # explode() handles lists of IDs if present, nunique() counts unique values
    unique_ids_count = annot_df['uniprot_ids'].explode().nunique()

    print(f"Number of unique UniProt IDs: {unique_ids_count}")
    def has_matching_id(ids):
        # Handle lists/arrays of IDs or single values
        if isinstance(ids, (list, np.ndarray)):
            return not unique_file_ids_set.isdisjoint(ids)
        return ids in unique_file_ids_set

    matched_annot_df = annot_df[annot_df['uniprot_ids'].apply(has_matching_id)]
    print(f"Found {len(matched_annot_df)} matching entries in annotation table")
    matched_annot_df.head()

else: 
    matched_annot_df = pd.read_parquet("data/matched_annotation_table.parquet")
    print(f"Loaded matched annotation table with {len(matched_annot_df)} entries")


Found 32714 matching entries in annotation table


,entry_pdb_id,entry_release_date,entry_oligomeric_state,entry_determination_method,entry_keywords,entry_pH,entry_resolution,entry_validation_resolution,entry_validation_rfree,entry_validation_r,...,system_protein_chains_total_length,system_unique_ccd_codes,system_proper_unique_ccd_codes,pdb_id,receptor_chain,ligand_chain,assembly_id,pdb_chain_key,uniprot_ids,uniprot_ids_str
0,8grd,2022-09-01,dimeric,X-RAY DIFFRACTION,OXIDOREDUCTASE,5.1,2.699,2.70,0.2418,0.1940,...,691,ADP,ADP,8grd,B,1.D,1,8grd_B,[O43837],O43837
9,4grb,2012-08-24,monomeric,X-RAY DIFFRACTION,Transferase/Transferase Inhibitor,6.5,2.150,2.15,0.2500,0.2200,...,333,0XG,0XG,4grb,A,1.C,1,4grb_A,[P68400],P68400
12,6gra,2018-06-10,monomeric,X-RAY DIFFRACTION,TRANSFERASE,7.5,2.600,2.60,0.2700,0.2100,...,285,F8Z,F8Z,6gra,A,1.B,1,6gra_A,[O14965],O14965
14,5grn,2016-08-11,monomeric,X-RAY DIFFRACTION,TRANSFERASE/TRANSFERASE INHIBITOR,,1.770,1.77,0.1793,0.1570,...,356,748,748,5grn,A,1.B,1,5grn_A,[P16234],P16234
16,4gra,2012-08-24,monomeric,X-RAY DIFFRACTION,TRANSFERASE,8.0,2.560,2.56,0.2661,0.2136,...,299,A3P,A3P,4gra,A,1.C,1,4gra_A,[P50225],P50225


In [4]:
matched_annotation_table['system_id']

NameError: name 'matched_annotation_table' is not defined

In [22]:
target_id = "O43837"
# Filter for rows containing the specific UniProt ID
specific_entries = annot_df[annot_df['uniprot_ids'].apply(
    lambda x: target_id in x if isinstance(x, (list, np.ndarray)) else x == target_id
)]
specific_entries

,entry_pdb_id,entry_release_date,entry_oligomeric_state,entry_determination_method,entry_keywords,entry_pH,entry_resolution,entry_validation_resolution,entry_validation_rfree,entry_validation_r,...,system_protein_chains_total_length,system_unique_ccd_codes,system_proper_unique_ccd_codes,pdb_id,receptor_chain,ligand_chain,assembly_id,pdb_chain_key,uniprot_ids,uniprot_ids_str
0,8grd,2022-09-01,dimeric,X-RAY DIFFRACTION,OXIDOREDUCTASE,5.1,2.699,2.7,0.2418,0.194,...,691,ADP,ADP,8grd,B,1.D,1,8grd_B,[O43837],O43837


In [33]:
# Check for ligand-related columns, especially SMILES
ligand_cols = [c for c in annot_df.columns if 'ligand' in c.lower() or 'smiles' in c.lower()]
print(f"Found {len(ligand_cols)} ligand-related columns.")
# Print first few to see if we have SMILES
print(ligand_cols[:20])

# Check if we have a specific SMILES column
smiles_col = next((c for c in ligand_cols if 'smiles' in c), None)
if smiles_col:
    print(f"\nFound SMILES column: {smiles_col}")
    print(annot_df[smiles_col].head())
else:
    print("\nNo explicit 'smiles' column found in the filtered columns.")

Found 329 ligand-related columns.
['system_ligand_chains', 'system_num_covalent_ligands', 'system_proper_num_covalent_ligands', 'system_num_ligand_chains', 'system_proper_num_ligand_chains', 'system_ligand_max_qed', 'system_ligand_max_molecular_weight', 'system_proper_ligand_max_molecular_weight', 'system_ligand_validation_num_residues', 'system_ligand_validation_num_processed_residues', 'system_ligand_validation_percent_processed_residues', 'system_ligand_validation_average_rsr', 'system_ligand_validation_average_rsrz', 'system_ligand_validation_average_rscc', 'system_ligand_validation_average_occupancy', 'system_ligand_validation_percent_rsr_under_threshold', 'system_ligand_validation_percent_rscc_over_threshold', 'system_ligand_validation_percent_occupancy_over_threshold', 'system_ligand_validation_average_b_factor', 'system_ligand_validation_unknown_residue_count']

Found SMILES column: ligand_smiles
0    Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO[P@](=O)(O)OP(=O...
1    Nc1ncnc2c1ncn2[C@@H]1O

In [37]:
# Load the affinity data for Q14004 to check for SMILES
affinity_file = 'data/chembl_parallel_enriched_combined/uniprot_Q14004/Q14004_chembl_activities.parquet'
if os.path.exists(affinity_file):
    affinity_df = pd.read_parquet(affinity_file)
    print("Affinity Data Columns:")
    print(affinity_df.columns)
    
    # Check for smiles
    aff_smiles_col = next((c for c in affinity_df.columns if 'smiles' in c.lower()), None)
    if aff_smiles_col:
        print(f"\nFound Affinity SMILES column: {aff_smiles_col}")
        print(affinity_df[aff_smiles_col].head())
else:
    print(f"File not found: {affinity_file}")

Affinity Data Columns:
Index(['action_type', 'activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type',
       'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units', 'upper_value', 'value',
       'assay_description_enriched', 'assay_type_descripti

In [44]:
# --- Linking PDB Entries to Affinity Data using InChIKey ---

# 1. Get the systems for this UniProt ID from the metadata
import json
from rdkit import Chem

def get_inchikey(smiles):
    if not isinstance(smiles, str):
        return None
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return Chem.MolToInchiKey(mol)
    except Exception:
        return None
    return None

metadata_file = 'data/chembl_parallel_enriched_combined/uniprot_Q14004/Q14004_metadata.json'
with open(metadata_file, 'r') as f:
    metadata = json.load(f)

related_system_ids = metadata.get('system_ids', [])
print(f"Systems related to Q14004 (from metadata): {related_system_ids}")

# 2. Filter annot_df for these systems
related_structures = annot_df[annot_df['system_id'].isin(related_system_ids)]
print(f"Found {len(related_structures)} rows in annot_df matching these systems.")

# 3. Check for matches using InChIKey (more robust than SMILES string match)
print("Generating InChIKeys for affinity data...")
affinity_inchikeys = set()
# Get unique SMILES and convert to InChIKey
for smiles in affinity_df['canonical_smiles'].dropna().unique():
    ikey = get_inchikey(smiles)
    if ikey:
        affinity_inchikeys.add(ikey)

print(f"Generated {len(affinity_inchikeys)} unique InChIKeys from affinity data.")

matches = []
for idx, row in related_structures.iterrows():
    pdb_smiles = row.get('ligand_smiles')
    pdb_inchikey = get_inchikey(pdb_smiles)
    
    if pdb_inchikey and pdb_inchikey in affinity_inchikeys:
        matches.append({
            'system_id': row['system_id'],
            'ligand_smiles': pdb_smiles,
            'ligand_inchikey': pdb_inchikey,
            'match_type': 'InChIKey match'
        })

print(f"\nFound {len(matches)} structures with InChIKey match in affinity data.")
if matches:
    print(pd.DataFrame(matches))
else:
    print("No InChIKey matches found.")

Systems related to Q14004 (from metadata): ['7nxj__1__1.A__1.E', '7nxj__2__1.C__1.F']
Found 2 rows in annot_df matching these systems.
Generating InChIKeys for affinity data...
Generated 393 unique InChIKeys from affinity data.

Found 0 structures with InChIKey match in affinity data.
No InChIKey matches found.


In [31]:
matched_annot_df['entry_pdb_id'].nunique()

14061

In [ ]:
# Ensure get_inchikey is available
def get_inchikey_safe(smiles):
    if not isinstance(smiles, str):
        return None
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return Chem.MolToInchiKey(mol)
    except Exception:
        return None
    return None

print("Starting global ligand matching analysis...")

# 1. Prepare Annotation Data
# Calculate InChIKeys for the unique ligands in annot_df first (optimization)
print("Calculating InChIKeys for annotation ligands...")
# Create a copy to avoid SettingWithCopy warnings
annot_df_w_inchi = annot_df.copy()

# Use tqdm for progress on apply
tqdm.pandas(desc="Calculating InChIKeys")
annot_df_w_inchi['ligand_inchikey'] = annot_df_w_inchi['ligand_smiles'].progress_apply(get_inchikey_safe)

# Drop rows where InChIKey could not be generated
annot_valid = annot_df_w_inchi.dropna(subset=['ligand_inchikey'])
print(f"Valid ligands with InChIKeys: {len(annot_valid)} / {len(annot_df)}")

# Explode to handle multiple UniProt IDs
print("Exploding annotation dataframe by UniProt ID...")
annot_exploded = annot_valid.explode('uniprot_ids')

# 2. Identify available affinity files
affinity_dir = Path('data/chembl_parallel_enriched_combined')
# Check if combined directory exists, otherwise fall back to the one used earlier
if not affinity_dir.exists():
    print(f"Combined directory {affinity_dir} not found. Checking 'data/chembl_parallel_enriched'...")
    affinity_dir = Path('data/chembl_parallel_enriched')

affinity_files = list(affinity_dir.glob('uniprot_*/*_chembl_activities.parquet'))
print(f"Found {len(affinity_files)} affinity files.")

# Filter annotation data to only include UniProt IDs we have files for
available_uniprots = set(f.parent.name.split('_')[1] for f in affinity_files)
annot_subset = annot_exploded[annot_exploded['uniprot_ids'].isin(available_uniprots)]
print(f"Annotation entries matching available affinity files: {len(annot_subset)}")

# Group for fast lookup
annot_grouped = annot_subset.groupby('uniprot_ids')

# 3. Iterate and Match
all_matches = []
print("Scanning affinity files for matches...")

# Iterate through files with a progress bar
for file_path in tqdm(affinity_files, desc="Processing Affinity Files"):
    try:
        uniprot_id = file_path.parent.name.split('_')[1]
        
        # Skip if no corresponding annotation entries
        if uniprot_id not in annot_grouped.groups:
            continue
            
        # Get relevant annotation rows
        relevant_annot = annot_grouped.get_group(uniprot_id)
        
        # Load affinity data
        aff_df = pd.read_parquet(file_path)
        if 'canonical_smiles' not in aff_df.columns:
            continue
            
        # Get unique SMILES from affinity data
        aff_smiles = aff_df['canonical_smiles'].dropna().unique()
        
        # Calculate InChIKeys for affinity ligands
        aff_inchikeys = set()
        for s in aff_smiles:
            k = get_inchikey_safe(s)
            if k:
                aff_inchikeys.add(k)
                
        # Find matches: Check which rows in relevant_annot have a ligand_inchikey in aff_inchikeys
        # We use set intersection for speed
        relevant_inchikeys = set(relevant_annot['ligand_inchikey'])
        matching_keys = relevant_inchikeys.intersection(aff_inchikeys)
        
        if matching_keys:
            # Retrieve the full rows for the matching keys
            matches = relevant_annot[relevant_annot['ligand_inchikey'].isin(matching_keys)]
            
            for _, match_row in matches.iterrows():
                all_matches.append({
                    'uniprot_id': uniprot_id,
                    'system_id': match_row['system_id'],
                    'ligand_smiles': match_row['ligand_smiles'],
                    'ligand_inchikey': match_row['ligand_inchikey'],
                    'affinity_file': str(file_path)
                })
                
    except Exception as e:
        # print(f"Error processing {file_path}: {e}")
        continue

# 4. Report and Save
matches_df = pd.DataFrame(all_matches)
print(f"\nTotal matches found: {len(matches_df)}")

if not matches_df.empty:
    print(f"Unique systems with matches: {matches_df['system_id'].nunique()}")
    print(f"Unique UniProt IDs with matches: {matches_df['uniprot_id'].nunique()}")
    
    output_path = 'data/ligand_matches_inchikey.csv'
    matches_df.to_csv(output_path, index=False)
    print(f"Matches saved to {output_path}")
    print(matches_df.head())
else:
    print("No matches found across the dataset.")

Starting global ligand matching analysis...
Calculating InChIKeys for annotation ligands...


Calculating InChIKeys:   0%|          | 0/114537 [00:00<?, ?it/s]

[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] Invalid InChI prefix in generating InChI Key
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] Invalid InChI prefix in generating InChI Key
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] Invalid InChI prefix in generating InChI Key
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] bond type above 3 (17) is treated as unspecified!
[16:07:09] Invalid InChI prefix in gene

Valid ligands with InChIKeys: 114537 / 114537
Exploding annotation dataframe by UniProt ID...
Found 1387 affinity files.
Annotation entries matching available affinity files: 32834
Scanning affinity files for matches...


Processing Affinity Files:   0%|          | 0/1387 [00:00<?, ?it/s]


Total matches found: 8862
Unique systems with matches: 8511
Unique UniProt IDs with matches: 663
Matches saved to data/ligand_matches_inchikey.csv
  uniprot_id          system_id  \
0     P27707  4q1f__1__1.A__1.D   
1     P27707  4q1f__1__1.B__1.F   
2     P27707  4q1b__1__1.A__1.C   
3     P27707  4q1b__1__1.B__1.E   
4     P27707  4q1a__1__1.A__1.C   

                                       ligand_smiles  \
0  COc1ccc(-c2nc([C@@H](C)Sc3nc(N)cc(N)n3)c(C)s2)...   
1  COc1ccc(-c2nc([C@@H](C)Sc3nc(N)cc(N)n3)c(C)s2)...   
2  CCCc1sc(-c2cccc(OCCNS(C)(=O)=O)c2)nc1CSc1nc(N)...   
3  CCCc1sc(-c2cccc(OCCNS(C)(=O)=O)c2)nc1CSc1nc(N)...   
4    CCCc1sc(-c2ccc(OC)c(OCCO)c2)nc1CSc1nc(N)cc(N)n1   

               ligand_inchikey  \
0  MBPWFMBRNJJXFY-GFCCVEGCSA-N   
1  MBPWFMBRNJJXFY-GFCCVEGCSA-N   
2  ULNBDBXWVRBZMV-UHFFFAOYSA-N   
3  ULNBDBXWVRBZMV-UHFFFAOYSA-N   
4  IFMIBDOPTYIDDA-UHFFFAOYSA-N   

                                       affinity_file  
0  data/chembl_parallel_enriched_combined/un

# Construct the database

In [47]:
# --- Create Clean Affinity Directory ---
import shutil

source_dir = Path('data/chembl_parallel_enriched')
dest_dir = Path('data/chembl_affinity')

print(f"Source: {source_dir}")
print(f"Destination: {dest_dir}")

if not dest_dir.exists():
    dest_dir.mkdir(parents=True)
    print(f"Created {dest_dir}")
else:
    print(f"Destination {dest_dir} already exists.")

# Get all uniprot directories
uniprot_dirs = list(source_dir.glob('uniprot_*'))
print(f"Found {len(uniprot_dirs)} UniProt directories to check.")

copied_count = 0
skipped_empty = 0
skipped_error = 0

for u_dir in tqdm(uniprot_dirs, desc="Filtering and Copying"):
    try:
        # Find the parquet file
        parquet_files = list(u_dir.glob('*_chembl_activities.parquet'))
        if not parquet_files:
            continue
            
        # Assume one main activity file per folder
        pq_file = parquet_files[0]
        
        # Check if empty
        try:
            df = pd.read_parquet(pq_file)
            if len(df) == 0:
                skipped_empty += 1
                continue
        except Exception:
            skipped_error += 1
            continue
            
        # If we get here, it's valid and non-empty
        # Create dest folder
        dest_u_dir = dest_dir / u_dir.name
        dest_u_dir.mkdir(exist_ok=True)
        
        # Copy parquet
        shutil.copy2(pq_file, dest_u_dir / pq_file.name)
        
        # Copy metadata if exists
        json_files = list(u_dir.glob('*.json'))
        for jf in json_files:
            shutil.copy2(jf, dest_u_dir / jf.name)
            
        copied_count += 1
        
    except Exception as e:
        print(f"Error processing {u_dir}: {e}")
        skipped_error += 1

print(f"\nOperation Complete.")
print(f"Copied (Non-empty): {copied_count}")
print(f"Skipped (Empty): {skipped_empty}")
print(f"Skipped (Error/No file): {skipped_error}")

Source: data/chembl_parallel_enriched
Destination: data/chembl_affinity
Created data/chembl_affinity
Found 1387 UniProt directories to check.


Filtering and Copying:   0%|          | 0/1387 [00:00<?, ?it/s]


Operation Complete.
Copied (Non-empty): 1387
Skipped (Empty): 0
Skipped (Error/No file): 0
